### Table of Contents 目录

* [安装数据及建立环境](#chapter1)
* [创建函数](#chapter2)
    * [根据姓+字+朝代（optional）查询人物](#section_2_1)
    * [根据姓+官职+朝代（optional）查询人物](#section_2_2)
* [调用函数](#chapter3)
    * [根据姓+字查询人物](#section_3_1)
    * [根据姓+字+朝代查询人物](#section_3_2)
    * [根据姓+官职查询人物](#section_3_3)
    * [根据姓+官职+朝代查询人物](#section_3_4)

        

<a name="chapter1"></a>
### 安装数据及建立环境 



In [ ]:
# Download CBDB sqlite
!git clone https://github.com/cbdb-project/cbdb_sqlite.git

Cloning into 'cbdb_sqlite'...
remote: Enumerating objects: 12, done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 12
Unpacking objects: 100% (12/12), done.


In [ ]:
# Explode cbdb_20201110.7z
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('./cbdb_sqlite/cbdb_20201110.7z').extractall(".")

     |████████████████████████████████| 81kB 5.3MB/s 


In [ ]:
# Create connection to database
import sqlite3
global CONN
CONN = sqlite3.connect("./cbdb_20201110.db")

In [ ]:
!pip install opencc

<a name="chapter2"></a>
### 创造函数 


<a name="section_2_1"></a>
### 根据姓+字+朝代查询人物（optional）

In [ ]:
# Function to search for a person based on surname and alternative name (e.g. 字)
# Dynasty is optional. If not passed in, all results will be returned.
import opencc
s2t_converter = opencc.OpenCC('s2t.json')
def get_person_by_surname_and_altname(surname, altname, dynasty=None):
    surname = s2t_converter.convert(surname)
    altname = s2t_converter.convert(altname)
    if dynasty:
      dynasty = s2t_converter.convert(dynasty)

    cur = CONN.cursor()
    if dynasty:
      sql_statement = "select * from (select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='" + altname + "' and BIOG_MAIN.c_surname_chn= '" + surname +"') tmp_db LEFT JOIN DYNASTIES on tmp_db.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='" + dynasty + "';"
    else:
      sql_statement = "select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='" + altname + "' and BIOG_MAIN.c_surname_chn='" + surname + "';"
    print(sql_statement)
    cur.execute(sql_statement)
    rows = cur.fetchall()
    count = 0
    for row in rows:
      count += 1
      print("Record"+str(count)+": ")
      print(row)

<a name="section_2_2"></a>
### 根据姓+官职+朝代（optional）查询人物

In [ ]:
def get_person_by_surname_and_office(surname, office, dynasty=None):
    surname = s2t_converter.convert(surname)
    office = s2t_converter.convert(office)
    if dynasty:
      dynasty = s2t_converter.convert(dynasty)
    cur = CONN.cursor()
    if dynasty:
      sql_statement = "select * from (select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%" + office + "%' or OFFICE_CODES.c_office_chn like '%" + office + "%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '" + surname +"') tmp_db2 LEFT JOIN DYNASTIES on tmp_db2.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='" + dynasty +"';"  
    else:
      sql_statement = "select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%" + office + "%' or OFFICE_CODES.c_office_chn like '%" + office + "%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '" + surname + "';"
    print(sql_statement)
    cur.execute(sql_statement)
    rows = cur.fetchall()
    count = 0
    for row in rows:
      count += 1
      print("Record"+str(count)+": ")
      print(row)

<a name="chapter3"></a>
### 调用函数 

<a name="section_3_1"></a>
### 根据姓+字查询人物

In [ ]:
# Example: Get person by surname + altname
surname = "吕"
zi = "留卿"
get_person_by_surname_and_altname(surname, zi)

select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='留卿' and BIOG_MAIN.c_surname_chn='呂';
Record1: 
(14990, 20662, 'Lv Shizhang', '呂師張', None, None, 20657, 0, 0, 0, None, None, 0, None, None, None, 0, None, None, None, None, 1291, 623, 28, '至元二10八年任衢州路同知', None, 0, None, None, 'Lu9', '呂', 'Shizhang', '師張', 18, 0, 'Lu(9) Shizhang [20662] WDY, 1, 424.', 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, 'TTS', '20070312', None, None, 1, 8478, 20662, 'Liuqing', '留卿', 4, 9599, '2590', None, 'load', '20110330', None, None)


<a name="section_3_1"></a>
### 根据姓+字+朝代查询人物 

In [ ]:
# Example: Get person by surname + altname + dynasty
surname = "吕"
zi = "留卿"
dynasty = "元"
get_person_by_surname_and_altname(surname, zi, dynasty)

select * from (select * from BIOG_MAIN INNER JOIN ALTNAME_DATA on BIOG_MAIN.c_personid=ALTNAME_DATA.c_personid where ALTNAME_DATA.c_alt_name_chn='留卿' and BIOG_MAIN.c_surname_chn= '呂') tmp_db LEFT JOIN DYNASTIES on tmp_db.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='元';
Record1: 
(14990, 20662, 'Lv Shizhang', '呂師張', None, None, 20657, 0, 0, 0, None, None, 0, None, None, None, 0, None, None, None, None, 1291, 623, 28, '至元二10八年任衢州路同知', None, 0, None, None, 'Lu9', '呂', 'Shizhang', '師張', 18, 0, 'Lu(9) Shizhang [20662] WDY, 1, 424.', 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, 'TTS', '20070312', None, None, 1, 8478, 20662, 'Liuqing', '留卿', 4, 9599, '2590', None, 'load', '20110330', None, None, 18, 'Yuan', '元', 1234, 1367, 71)


<a name="section_3_3"></a>
### 根据根据姓+官职查询人物

In [ ]:
# Example: Get person by surname + office
surname = "张"
office = "平章"
get_person_by_surname_and_office(surname, office)

select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%平章%' or OFFICE_CODES.c_office_chn like '%平章%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '張';
Record1: 
(183, '中書門下平章事;門下平章;同平章事;平章事', '同中書門下平章事', 182, 183, 'Zhang Qixian', '張齊賢', 943, '01', None, 0, 0, 0, None, 943, 0, None, None, 1014, 515, 7, None, 72, None, None, 0, None, None, None, 0, None, None, 'Zhang', '張', 'Qixian', '齊賢', 15, 0, "Zhang(1) Qixian [183] His family moved to Luoyang when he was 3 because of disorder during the Five Dynasties. Zonghui's [305] and Zongjian's [3167] father, Zigao's [7171] grandfather. Father-in-law of Wang(2) Juzheng's [8145] brother, Jushan [3945]. Jushan's son-in-law was Sheng(1) Du's [8109] elder cousin Jing's [3739] son, Zunfu [3743]. Zhang(1

<a name="section_3_4"></a>
### 根据姓+官职+朝代查询人物

In [ ]:
# Example: Get person by surname + office + dynasty
surname = "张"
office = "平章"
dynasty = "唐"
get_person_by_surname_and_office(surname, office, dynasty)

select * from (select * from (select distinct(c_personid), OFFICE_CODES.c_office_chn_alt, OFFICE_CODES.c_office_chn from POSTED_TO_OFFICE_DATA JOIN OFFICE_CODES on POSTED_TO_OFFICE_DATA.c_office_id=OFFICE_CODES.c_office_id where OFFICE_CODES.c_office_chn_alt like '%平章%' or OFFICE_CODES.c_office_chn like '%平章%') tmp_db LEFT JOIN BIOG_MAIN on BIOG_MAIN.c_personid=tmp_db.c_personid where BIOG_MAIN.c_surname_chn= '張') tmp_db2 LEFT JOIN DYNASTIES on tmp_db2.c_dy=DYNASTIES.c_dy where DYNASTIES.c_dynasty_chn='唐';
Record1: 
(3130, None, '同中書門下平章事', 2413, 3130, 'Zhang Jiuling', '張九齡', 678, '01', None, 0, 0, 0, None, 678, 361, 3, None, 740, 375, 28, None, 63, None, 718, 375, 6, '左補闕（拜命 開元6(718)）', 737, 375, 25, '荊州長史（拜命 開元25(737)）', 'Zhang', '張', 'Jiuling', '九齡', 6, 72, 'Zhang(1) Jiuling [3130] Giles, 17-18. 《唐代人物知識ベース》記其生卒年為：678 - 740.', 0, 0, None, None, None, None, None, None, None, None, None, None, None, None, 'TTS', '20070312', None, None, 0, 6, 'Tang', '唐', 618, 907, 46)
Record2: 
(12419,